In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from PIL import Image
from matplotlib import rc

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

#nlp
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from selenium import webdriver
from tqdm.notebook import tqdm
import time, urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
keys = Keys()
import re, os, sys, json

from hanspell import spell_checker

from konlpy.tag import Kkma #형태소분석
from konlpy.tag import Okt #형태소분석
import soynlp; from soynlp.normalizer import * #정규화
# from pykospacing import Spacing------------------>써야함!
# spacing = Spacing()------------------------------>써야함!

from konlpy.tag import Komoran, Hannanum, Kkma, Okt
komoran = Komoran(); hannanum = Hannanum(); kkma = Kkma(); okt = Okt();

In [24]:
df_figure.iloc[: , 1:].to_csv('data/hoodtrain_sample.csv',index=False,encoding = "UTF-8")

In [61]:
df1 = pd.read_csv('data/hood_2_1_11page.csv')
df2 = pd.read_csv('data/hood_2_12_30page.csv')
df3 = pd.read_csv('data/hood_2_31_40page.csv')
df4 = pd.read_csv('data/hood_2_41_70page.csv')
df4.drop(['허리단면','허벅지단면','밑위','밑단단면'],axis = 1,inplace=True)
df5 = pd.read_csv('data/hood_2_71_90page.csv')
dfpage2 = pd.concat([df1,df2,df3,df4,df5])
dfpage2.to_csv('data/hood_2page.csv',encoding="UTF-8", index=False)



## 전처리에 들어갈 코드

* 사이즈 사전완성하고 돌릴 df완성

In [3]:
def crawlingdataconcat():
    df = pd.DataFrame()
    for i in range(1,21,1):
        try:
            df_before = pd.read_csv('data/hood_'+str(i)+'page.csv') #크롤링파일 불러오기
            print('df',str(i),': ',df_before.shape,end = ',')
        except:
            df_before = pd.DataFrame() #아직크롤링되지 않은파일은 빈 df
            print('df',str(i),': ',df_before.shape)
            
        df = pd.concat([df,df_before]) #합치기
            

            
    print('모두합친df : ', df.shape)
    df.drop_duplicates(subset = None,keep = 'first', inplace = True,ignore_index = True)
    print('중복제거df : ', df.shape)#전체열이 같은 중복제거
    #성별, 키,몸무게 모두 null아니고 수치종류 적어도 하나 값이 있는 것들 filter
    df = df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())]
    print('최종사용 data:{}'.format(df.shape))
    print('-'*10)
    return df

def crawlingdataprocessing(df):
    #kg,cm제거 및 수치타입으로 변경
    df["height"]=df["height"].replace('cm','',regex = True)
    df["weight"]=df["weight"].replace('kg','',regex = True)
    df= df.astype({'height':'float','weight':'float'})
    print('df전처리완료')
    print('-'*10)
    return df

def add_reviewcol(df):
    df = df.drop_duplicates(subset=['content'])#리뷰중복제거
    print('같은내용리뷰 중복제거 완료')
    df.reset_index(drop=True, inplace=True)

    df['review'] = str('')
    #외국어 리뷰 삭제
    i = 0; nohangul = []
    for i in range(df.shape[0]):
        text = re.sub('[^ㄱ-ㅣ가-힣]', '',df.iloc[i,8])
        if(text==''):
            nohangul.append(i)
    df = df.iloc[[True if i not in nohangul else False for i in range(df.shape[0])],:]
    df.reset_index(drop=True, inplace=True)

    i=0
    for i in range(df.shape[0]):
        text = df.iloc[i,7]
        text = re.sub(pattern='[^\w\s\n]', repl='', string=text) #특수문자 제거
        text = re.sub(pattern='[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z]', repl=' ', string=text) #숫자, 그이외 삭제
        text = re.sub(pattern='[ㄱ-ㅎㅏ-ㅣ]+', repl='', string=text) #단순 모음, 자음 삭제
        text = repeat_normalize(text, num_repeats=2) #불필요반복문자정규화
        #text = spacing(text) #띄어쓰기------------------------>써야함!
        df['review'][i] = text
    print('리뷰 전처리 열 추가')
    print('df shape:{}'.format(df.shape))
    print('-'*10)
    return df

In [54]:
#최종 data = df
df = crawlingdataconcat()
df = crawlingdataprocessing(df)
df_before_nlp = add_reviewcol(df)

df 1 :  (9000, 17),df 2 :  (8808, 17),df 3 :  (8370, 17),df 4 :  (6690, 17),df 5 :  (7649, 17),df 6 :  (6419, 17),df 7 :  (5687, 17),df 8 :  (5715, 17),df 9 :  (5102, 17),df 10 :  (5154, 17),df 11 :  (4318, 17),df 12 :  (3671, 17),df 13 :  (3389, 17),df 14 :  (3673, 17),df 15 :  (3103, 17),df 16 :  (2906, 17),df 17 :  (2658, 17),df 18 :  (2445, 17),df 19 :  (1903, 17),df 20 :  (1734, 17),모두합친df :  (98394, 17)
중복제거df :  (39203, 17)
최종사용 data:(22502, 17)
----------
df전처리완료
----------
같은내용리뷰 중복제거 완료
리뷰 전처리 열 추가
df shape:(20037, 18)
----------


In [39]:
df_before_nlp.head(2)

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,어깨너비,가슴단면,소매길이,review
0,LV 3 뉴비_e05c4c870ff1,여성,160.0,42.0,SIGN LOGO HOOD GRAY,M,5.0,면이 톡톡해서 좋아요. 너무 얇지도 두껍지도 않아요...평상시 S입는데 오버핏으로 ...,보통이에요,보통이에요,선명해요,보통이에요,M,70.0,63.0,59.0,55.0,면이 톡톡해서 좋아요 너무 얇지도 두껍지도 않아요평상시 S입는데 오버핏으로 입고 싶...
1,LV 4 배에앵,남성,174.0,78.0,SIGN LOGO HOOD GRAY,L,5.0,자주사용할 회색후드티 찾고있었는데 가성비도 좋고 기본템으로 활용하기 좋아 구매했어요!,보통이에요,보통이에요,보통이에요,보통이에요,L,72.0,64.0,61.0,56.0,자주사용할 회색후드티 찾고있었는데 가성비도 좋고 기본템으로 활용하기 좋아 구매했어요


### 사이즈사전

In [5]:
#total,chongjang,shoulder,chest,arm,small,big
total = ['사이즈','사이즈하','사이즈감','임사이즈','여사이즈','사이즌','뻐요사이즈','싸이즈','핏이','핏','핏입니다',
      '핏감도','핏나','핏감이랑','핏처럼','핏입니당','핏이에요','핏이예요','핏이랑','핏이구요','핏이네요','핏감은',
      '핏입니','핏이라서','핏으로','옷핏','핏더','핏도','핏감','옷','후드','후드티','핏은','좋아요핏','요핏도',
      '좋아요핏도','상의','요핏','폼','폭','사이즈','후드','옷','품','전체','품도','몸통','옷핏','상체','핏',
      '핏이구','핏이었','핏임','핏입',
      '핏입니','핏감','요핏','폼','후드핏','핏이','크기','상품','상의','핏더']
chongjang = ['길이','기장','총장','궁디','궁딩','엉덩이','밑','밑단','위아래','끝단','밑기장',
             '밑단이랑','기장이','기장입니','밑단','밑','기장감','요기장','총길이','총장','총기장',
            '길이감','끝','기장','길이']
shoulder = ['어깨핏','어께','어깨','너비','어깡','골격','바디','가로','넓이','등판','통',
            '어깨깡패','어깨라','어깨넓','어깨쪽','어께','몸집','어깨핏','어깨선','어깨라인','어깡',
            '어깨','통','몸통','골격','넓이','너비','등판']

chest = ['가슴','둘레','바디','가로','넓이','통','둘레','몸집','가슴팍','통','몸통']
arm = ['소매','팔도','손목','손','팔다리','팔목','팔','팔꿈치','팔이','요팔',
       '당팔','팔길이','팔다리','팔길','팔소매','손목','팔목','소매통','손','팔','소매','팔기장']

small = ['작습니다','짧았어요','짧습니다','짧으면','짧','작았어요','짧다고','쪼여서','작았으면',
      '작아도','작으면','작음','작아진','좁다고',
      '짧아요','짧게','짧다는','좁다는','작네','작기는','달라붙는','작은데','짧은데','짧음',
      '작긴','짧네요','작게','작아서','작은','짧고','짧아','좁은','타이트','짧았지만','붙어서','짧긴',
      '작은것','작아요','짧은','짧아서','짧지만','좁아','좁고','작네요','작아','쪼이는','좁아요','크롭느낌',
      '크롭하','작고','숏','쫄려','작다고','작다','좁아서','작','크롭','크롭된','미니','크롭해',
      '길었으면','컸으면','좁게','짧지' ,'작지','크롭이','핏되',
      '숏하','사이즈업하','크롭하','숏한','크롭하긴','크롭이','크롭해','크롭느낌','크롭된','사이즈업','짝',
       '작','숏','짧긴','크롭','크롭한','타이트','크롭이라','작긴',
       '작','짧','좁']

big = ['큰','긴','커요','크고','크긴','넉넉하고','길어서','넉넉하게','작다는'
     '넉넉한','클','길고','큰데','크지','크지만','부해','크니깐','크더라구요','컸어도','넓음','큼직하게',
     '박시하네요','넉넉할','헐렁하게','큼직하니','컸음','큽니','긴데','길어','길게','넓어서','넓어','박시하고','컸어요',
     '박시','크다고','크다는','넓게','넓고','커용','길다는','크지도','큼','크다','길었어요','넉넉함','큰데도','덮는데',
     '덮이고','컸고','벙벙해서','크더라고요','컸을','크거나','크더라','컸었는데','컸어용','컸습니다','덮네요',
     '큼지막해서','덥는','박시하게','크네요','클까','넉넉하구','박시하','컸는데','넉넉하네요','크게','크니까','넉넉한데','길어요',
     '컸지만','헐렁한','크네','넉넉해','펑퍼짐한','덮어용','박시하니','헐렁해','넉넉하지''넉넉하구요','커','큽니다','넉넉하니',
     '크','길지','박시한데','덮어요','길긴','넉넉해요','넓은','덮히는', '넓긴','덮어줘서','덮은','넉넉해서','큼직해서','기네','낙낙',
     '흘러내리는','덮어서','벙벙한','커서','큰것','박시핏','오퍼핏','낙낙해','여유핏','널널함','길','덮어','넉넉합니다','크면',
     '덮고','가오리','루즈핏','펑퍼짐','길었지만','헐렁하고','남아','아방핏','박스핏','널널','오바핏','덮습니다','넉넉히',
     '헐렁해서','접어도','접어','접어서','널','넓어요','덮고도','오버','와이드','낭낭','헐렁','접으면','짧았으면','접고',
     '버핏','접어야','잡아먹힌','수선해서','오버핏','오버사이즈','오버사이징해서','낙낙하다','낙낙한데','낙낙해요','오버핏이예요','오버핏입니당',
       '요박시하','컸어용','낙낙합니','어벙벙하','와이드핏','크니깐','크네요옷','널널해요','오버핏나와요','널럴하',
       '요넉넉하','엄청큼','여유핏','오버느낌','아방핏','오버핏은','넓긴','넉넉함','입니다오버핏','큽니다오버핏',
       '큽니다제','큽니당','오버해요','요박시한','헐렁하긴','헐렁거리','아방방한','아방방','박시합니','박시했다',
       '빅사이즈','받았습니다오버핏','빡시하','루즈핏으로','낭낭한','오버핏이긴','박스핏','널널함','어벙벙',
       '요넉넉한','오퍼핏','오버핏이구요','낙낙해','낭낭해서','넉넉하구','넉넉한데','아방해','오버핏이라서',
       '낭낭하','박시해','커용','넉넉해','펑퍼짐','박시하지','널널','좋아요오버핏','어벙벙한','널널하','오바핏',
       '오버하지','오버핏하','박시합니다','오버핏되','박시함','버핏','박시한','낙낙하니','박시해요','크네용',
       '낙낙해서','큼직해서','루즈해','느슨','널널합니다','아방아','아방아방','루즈핏이','박시하','세미오버',
       '세미오버핏','루즈해서','오버핏이라','루즈핏이라','오버핏이네요','오버핏입니','어벙','오버해','박시한데',
       '아방','오버핏이에요','예뻐요오버핏','큼지막','박시핏','오버해서','루즈한','오버핏인데','넉넉하구요','박시할',
       '큼직','오버한','오버핏이','낙낙하','아방하','롱','오버핏으로','낙낙한','오버핏입니다','박시해서','와이드',
       '요오버핏','넉넉해서','큽니','오버','오버하','루즈','루즈핏','박시','넉넉','길','크긴','아방한','널널한',
       '헐렁','길긴','와이드하','아방해서','세미오버사이즈',             
       '크','길','널','덮','넓','벙벙하','덮이','덮히','커다랗','덥히','넉넉해지','뒤집'             
       '넉넉']

In [6]:
# 두개의 사전이 합치고 나서 진행
# 중복제거
# 'n+2'개의 최종 리스트가 생성됨
def sizedict_nodup(oneOfRawList):
    print('중복제거 전:{}개'.format(len(oneOfRawList)))
    oneOfRawList = set(oneOfRawList)
    print('중복제거 후:{}개'.format(len(oneOfRawList)))
    print('-'*10)
    return list(oneOfRawList)

In [7]:
total     = sizedict_nodup(total)
chongjang = sizedict_nodup(chongjang)
shoulder  = sizedict_nodup(shoulder)
arm       = sizedict_nodup(arm)
chest     = sizedict_nodup(chest)
small     = sizedict_nodup(small)
big       = sizedict_nodup(big)

중복제거 전:64개
중복제거 후:52개
----------
중복제거 전:25개
중복제거 후:20개
----------
중복제거 전:28개
중복제거 후:19개
----------
중복제거 전:22개
중복제거 후:16개
----------
중복제거 전:11개
중복제거 후:9개
----------
중복제거 전:89개
중복제거 후:76개
----------
중복제거 전:290개
중복제거 후:255개
----------


### 사전으로 01추출해보기 여기부터 해보기 마구잡이

In [55]:
from tqdm import tqdm
def get_keywords(keywords, df,keyword_column_name):
    start = time.time()
    review = df['review']
    df['new_column'] = str('')
    for i in tqdm(range(len(review))):
        # if i % 1000 ==0:
        #     print("{}번째 리뷰 완료(천 단위) : ".format(i))
        keywords_search = []
        for j in keywords:
            if re.findall(j, review[i]):
                a = re.findall(j +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i]) #키워드 +단어
                aa = re.findall(j + ' '+'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])#키워드 +띄고 단어
                b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+' + j ,review[i]) #단어 + 키워드
                bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + j ,review[i])#단어 + 띄고 키워드
                #bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' +' '+ j ,review[i])
                keywords_search.extend(a)
                keywords_search.extend(aa)
                keywords_search.extend(b)
                keywords_search.extend(bb)
                # print(i,'번째',j,a,end='|')
                # print(aa,end='|')
                # print(b,end='|')
                # print(bb)
        #print(i,'번째 리스트:',keywords_search)
        #print('-'*10)
        
        if len(keywords_search) != 0:
            keywords_o = ','.join(x for x in keywords_search)
            df['new_column'][i] = keywords_o 
            
        else:
            df['new_column'][i] = '0'
    
    a = df.rename(columns = {'new_column':keyword_column_name})
    end = time.time()
    print("{:.5f} sec".format(end-start))
    return a[keyword_column_name]

In [84]:
#total,chongjang,shoulder,chest,arm ->keywords리스트
keyword_column_name = 'total_keyword'
df_total            = get_keywords(total,df_before_nlp,keyword_column_name)
keyword_column_name = 'chongjang_keyword'
df_chongjang        = get_keywords(chongjang,df_before_nlp,keyword_column_name)
keyword_column_name = 'shoulder_keyword'
df_shoulder         = get_keywords(shoulder,df_before_nlp,keyword_column_name)
keyword_column_name = 'chest_keyword'
df_chest            = get_keywords(chest,df_before_nlp,keyword_column_name)
keyword_column_name = 'arm_keyword'
df_arm              = get_keywords(arm,df_before_nlp,keyword_column_name)

df_keywords         = pd.concat([df_before_nlp,
                                 df_total,
                                df_chongjang,
                                df_chest,
                                df_shoulder,
                                df_arm],axis = 1)
df_keywords =df_keywords.drop(['new_column'],axis = 1)
df_keywords_origin = df_keywords.copy()
#df_keywords.info()
# total = ['사이즈','사이즈하','사이즈감','임사이즈','여사이즈','사이즌','뻐요사이즈','싸이즈','핏이','핏','핏입니다',
#       '핏감도','핏나','핏감이랑','핏처럼','핏입니당','핏이에요','핏이예요','핏이랑','핏이구요','핏이네요','핏감은',
#       '핏입니','핏이라서','핏으로','옷핏','핏더','핏도','핏감','옷','후드','후드티','핏은','좋아요핏','요핏도',
#       '좋아요핏도','상의','요핏','폼','폭','사이즈','후드','옷','품','전체','품도','몸통','옷핏','상체','핏',
#       '핏이구','핏이었','핏임','핏입',
#       '핏입니','핏감','요핏','폼','후드핏','핏이','크기','상품','상의','핏더']
#0,1,5,7,9,11,12,13,15,17,18,19,

100%|██████████| 20037/20037 [00:15<00:00, 1321.13it/s]


15.18752 sec


100%|██████████| 20037/20037 [00:09<00:00, 2139.42it/s]


9.38559 sec


100%|██████████| 20037/20037 [00:09<00:00, 2180.91it/s]


9.21635 sec


100%|██████████| 20037/20037 [00:08<00:00, 2488.62it/s]


8.06842 sec


100%|██████████| 20037/20037 [00:08<00:00, 2286.54it/s]


8.77698 sec


In [66]:

print(df_keywords.info())
print('전체:{}, 총장:{},가슴:{},소매:{},어깨{}'.format(df_keywords[df_keywords['total_keyword']!='0'].shape,
                                             df_keywords[df_keywords['chongjang_keyword']!='0'].shape,
                                             df_keywords[df_keywords['chest_keyword']!='0'].shape,
                                             df_keywords[df_keywords['arm_keyword']!='0'].shape,
                                             df_keywords[df_keywords['shoulder_keyword']!='0'].shape))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20037 entries, 0 to 20036
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user               20037 non-null  object 
 1   gender             20037 non-null  object 
 2   height             20037 non-null  float64
 3   weight             20037 non-null  float64
 4   item               20037 non-null  object 
 5   size               20037 non-null  object 
 6   star               20037 non-null  float64
 7   content            20037 non-null  object 
 8   size_eval          20037 non-null  object 
 9   bright_eval        20037 non-null  object 
 10  color_eval         20037 non-null  object 
 11  thick_eval         20037 non-null  object 
 12  cm                 20037 non-null  object 
 13  총장                 20037 non-null  float64
 14  어깨너비               19214 non-null  float64
 15  가슴단면               20037 non-null  float64
 16  소매길이               200

In [67]:
df_keywords[df_keywords['chest_keyword']!='0'].head(5)

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,어깨너비,가슴단면,소매길이,review,total_keyword,chongjang_keyword,chest_keyword,shoulder_keyword,arm_keyword
106,LV 2 뉴비_0fe4fd2b7f40,남성,173.0,60.0,(23SS) 2 TONE ARCH HOODIE GRAY,M,5.0,사이즈 보통이고 밝기도 밝고 색감도 선명하고 두께감도 두꺼워서 입기 좋네요 이래서 ...,보통이에요,밝아요,선명해요,두꺼워요,M,67.0,58.0,59.0,62.0,사이즈 보통이고 밝기도 밝고 색감도 선명하고 두께감도 두꺼워서 입기 좋네요 이래서 ...,사이즈 보통이고 밝기도 밝고,0,"통이고 밝기도 밝고,보통","통이고 밝기도 밝고,보통",0
107,LV 2 뉴비_0fe4fd2b7f40,여성,163.0,60.0,(23SS) 2 TONE ARCH HOODIE GRAY,L,5.0,사이즈 보통이고 밝기도 밝고 색감도 선명하고 두께감도 두꺼워서 입기 좋아요 ㅎㅎ,보통이에요,밝아요,선명해요,두꺼워요,L,69.0,60.0,61.0,63.0,사이즈 보통이고 밝기도 밝고 색감도 선명하고 두께감도 두꺼워서 입기 좋아요,사이즈 보통이고 밝기도 밝고,0,"통이고 밝기도 밝고,보통","통이고 밝기도 밝고,보통",0
184,LV 5 쿠니하늘,남성,176.0,74.0,[Mmlg] 19MG HOOD (DENVER GREEN),L (기모X),4.0,원하던 색 : 완전 내(스타일)추구사이즈 : 상의 체형보정과 널널하게면 보통175-...,커요,보통이에요,보통이에요,보통이에요,L (기모X),70.0,59.0,66.0,67.0,원하던 색 완전 내스타일추구사이즈 상의 체형보정과 널널하게면 보통 까지는 L사이즈 ...,"사이즈 상의 체형보정과 널널하게면,사이즈 없음품질 너무 가벼워서,내스타일추구사이즈,...",0,보통,보통,0
243,LV 4 elly88,남성,178.0,75.0,[Mmlg] 19MG HOOD (REDBEAN),L,4.0,기모인데도 많이 두껍지 않아요팔길이는 좋은데 그에 비해 몸통 길이가 짧아서 시보리가...,보통이에요,보통이에요,보통이에요,보통이에요,L,70.0,59.0,66.0,67.0,기모인데도 많이 두껍지 않아요팔길이는 좋은데 그에 비해 몸통 길이가 짧아서 시보리가...,"몸통 길이가 짧아서 시보리가,비해 몸통,핏이 아쉬웠어요 그래도,튀어나와 핏,핏이 아...","길이는 좋은데 그에,길이가 짧아서 시보리가,않아요팔길이,몸통 길이","통 길이가 짧아서 시보리가,몸통,몸통 길이가 짧아서 시보리가,비해 몸통","몸통 길이가 짧아서 시보리가,비해 몸통,통 길이가 짧아서 시보리가,몸통","요팔길이는 좋은데 그에,않아요팔,팔길이는 좋은데 그에,않아요팔길이,팔길이는 좋은데 ..."
280,LV 3 미소천사YJHJNY,남성,170.0,75.0,미시간 울버린 피그먼트 후드티 블랙 차콜,[NO기모]LARGE,4.0,여름을 제외하고 입을거라 기모 없는 상품으로알아보던 중 하루특가 행사로 할인가로 구...,커요,보통이에요,보통이에요,보통이에요,[NO기모]LARGE,73.0,64.0,62.0,58.0,여름을 제외하고 입을거라 기모 없는 상품으로알아보던 중 하루특가 행사로 할인가로 구...,"상품으로알아보던 중 하루특가,상품평이 나쁘지 않아,없는 상품,후기와 상품,품으로알아...",0,"가로 구매 했네요구매 후기와,할인가로","가로 구매 했네요구매 후기와,할인가로",0


## big, small 감지

In [73]:
#small,big
def big_small(big,small,df,name_big_small):
    start = time.time()
    review = df[name_big_small+'_keyword']
    df[name_big_small + '_big'] = str('')
    df[name_big_small + '_small'] = str('')
    for i in tqdm(range(len(review))):
        # if i % 1000 ==0:
        #     print("{}번째 리뷰 완료(천 단위) : ".format(i))
        big_search = []
        small_search = []
        for j in big:
            if re.findall(j, review[i]):
                a = re.findall(j +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i]) #키워드 +단어
                aa = re.findall(j + ' '+'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])#키워드 +띄고 단어
                b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+' + j ,review[i]) #단어 + 키워드
                bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + j ,review[i])#단어 + 띄고 키워드
                #bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' +' '+ j ,review[i])
                big_search.extend(a)
                big_search.extend(aa)
                big_search.extend(b)
                big_search.extend(bb)
                time.sleep(1)
                # print(i,'번째',j,a,end='|')
                # print(aa,end='|')
                # print(b,end='|')
                # print(bb)
        #print(i,'번째 리스트:',keywords_search)
        #print('-'*10)
        for j in small:
            if re.findall(j, review[i]):
                a = re.findall(j +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i]) #키워드 +단어
                aa = re.findall(j + ' '+'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])#키워드 +띄고 단어
                b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+' + j ,review[i]) #단어 + 키워드
                bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + j ,review[i])#단어 + 띄고 키워드
                #bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' +' '+ j ,review[i])
                small_search.extend(a)
                small_search.extend(aa)
                small_search.extend(b)
                small_search.extend(bb)
                time.sleep(1)
                # print(i,'번째',j,a,end='|')
                # print(aa,end='|')
                # print(b,end='|')
                # print(bb)
        #print(i,'번째 리스트:',keywords_search)
        #print('-'*10)
        
        if len(big_search) != 0:
            df[name_big_small + '_big'][i] = 1
            
        else:
            df[name_big_small + '_big'][i] = 0
        
        if len(small_search) != 0:
            df[name_big_small + '_small'][i] = 1
            
        else:
            df[name_big_small + '_small'][i] = 0
    
    #df.rename(columns = {'keyword_column_name':keyword_column_name},inplace=True)
    end = time.time()
    print("{:.5f} sec".format(end-start))
    return df[[name_big_small + '_big',name_big_small + '_small']]

In [74]:
name_big_small      = 'total'
total_big_small     = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'chongjang'
chongjang_big_small = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'arm'
arm_big_small       = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'chest'
chest_big_small     = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'shoulder'
shoulder_big_small  = big_small(big,small,df_keywords,name_big_small)

final_df            = df_keywords
#df_keywords =df_keywords.drop(['new_column'],axis = 1)


100%|██████████| 20037/20037 [01:38<00:00, 202.51it/s]


98.94857 sec


100%|██████████| 20037/20037 [00:57<00:00, 347.25it/s]


57.70644 sec


100%|██████████| 20037/20037 [00:56<00:00, 357.60it/s]


56.03828 sec


100%|██████████| 20037/20037 [00:57<00:00, 346.05it/s]


57.90643 sec


100%|██████████| 20037/20037 [00:58<00:00, 341.51it/s]

58.67675 sec


In [79]:
print('전체big{},전체small{}'.format(final_df[final_df['total_big']!=0].shape,final_df[final_df['total_small']!=0].shape))
print('총장big{},총장small{}'.format(final_df[final_df['chongjang_big']!=0].shape,final_df[final_df['chongjang_small']!=0].shape))
print('가슴big{},가슴small{}'.format(final_df[final_df['chest_big']!=0].shape,final_df[final_df['chest_small']!=0].shape))
print('어깨big{},어깨small{}'.format(final_df[final_df['shoulder_big']!=0].shape,final_df[final_df['shoulder_small']!=0].shape))
print('소매big{},소매small{}'.format(final_df[final_df['arm_big']!=0].shape,final_df[final_df['arm_small']!=0].shape))


전체big(4183, 33),전체small(584, 33)
총장big(1279, 33),총장small(419, 33)
가슴big(154, 33),가슴small(42, 33)
어깨big(251, 33),어깨small(62, 33)
소매big(590, 33),소매small(68, 33)


In [80]:
final_df[final_df['chest_big']!=0].head(5)

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,어깨너비,가슴단면,소매길이,review,total_keyword,chongjang_keyword,chest_keyword,shoulder_keyword,arm_keyword,total_big,total_small,chongjang_big,chongjang_small,arm_big,arm_small,chest_big,chest_small,shoulder_big,shoulder_small
243,LV 4 elly88,남성,178.0,75.0,[Mmlg] 19MG HOOD (REDBEAN),L,4.0,기모인데도 많이 두껍지 않아요팔길이는 좋은데 그에 비해 몸통 길이가 짧아서 시보리가...,보통이에요,보통이에요,보통이에요,보통이에요,L,70.0,59.0,66.0,67.0,기모인데도 많이 두껍지 않아요팔길이는 좋은데 그에 비해 몸통 길이가 짧아서 시보리가...,"몸통 길이가 짧아서 시보리가,비해 몸통,핏이 아쉬웠어요 그래도,튀어나와 핏,핏이 아...","길이는 좋은데 그에,길이가 짧아서 시보리가,않아요팔길이,몸통 길이","통 길이가 짧아서 시보리가,몸통,몸통 길이가 짧아서 시보리가,비해 몸통","몸통 길이가 짧아서 시보리가,비해 몸통,통 길이가 짧아서 시보리가,몸통","요팔길이는 좋은데 그에,않아요팔,팔길이는 좋은데 그에,않아요팔길이,팔길이는 좋은데 ...",1,1,1,1,1,0,1,1,1,1
724,LV 4 556325df5032c,남성,176.0,81.0,어센틱 로고 후디 그레이,XL,5.0,오버핏사이즈입니다. 넓이 적당하고 길이도 길지 않아서 데일리룩에 적합합니다.,커요,보통이에요,보통이에요,보통이에요,XL,72.0,60.5,65.5,61.0,오버핏사이즈입니다 넓이 적당하고 길이도 길지 않아서 데일리룩에 적합합니다,"사이즈입니다 넓이 적당하고,오버핏사이즈,핏사이즈입니다 넓이 적당하고,오버핏","길이도 길지 않아서,적당하고 길이","넓이 적당하고 길이도 길지,오버핏사이즈입니다 넓이","넓이 적당하고 길이도 길지,오버핏사이즈입니다 넓이",0,1,0,1,0,0,0,1,0,1,0
874,LV 4 너나나나나나너,여성,165.0,60.0,후디드 스웨트셔츠 [멜란지 그레이],L,4.0,170에 65 . 가슴폭은 많이 크지않고 길이나 소매등 전체적으로 깔끔합니다. 조금...,보통이에요,보통이에요,보통이에요,보통이에요,L,70.0,NaN,57.0,89.0,에 가슴폭은 많이 크지않고 길이나 소매등 전체적으로 깔끔합니다 조금 무겁네요,"폭은 많이 크지않고,가슴폭,전체적으로 깔끔합니다 조금,소매등 전체","길이나 소매등 전체적으로,크지않고 길이","가슴폭은 많이 크지않고,에 가슴",0,"소매등 전체적으로 깔끔합니다,길이나 소매",1,0,1,0,0,0,1,0,0,0
897,LV 3 이동혁1banin여치니,여성,166.0,60.0,잇 저스트 마이 후디[그레이],M_쭈리,3.0,원래 라지 입는데 m 시켯거든어... 근데 진짜 너무너무너무너무 커서 솔직히 첨에 ...,커요,보통이에요,보통이에요,두꺼워요,M_쭈리,73.0,61.0,65.0,72.0,원래 라지 입는데 m 시켯거든어 근데 진짜 너무너무너무너무 커서 솔직히 첨에 널랬엇...,"핏이 바부같아졌어요 버렫야될,갑자기 핏,핏이 바부같아졌어요 버렫야될 것,갑자기 핏이",0,"통도 진짜 큰데,크고 통","통도 진짜 큰데,크고 통","팔 접고 입으니까 뭐,팔 진짜 크고 통도,근데 팔,괜찮드라구요 팔",0,0,0,0,1,0,1,0,1,0
1481,LV 8 하나만사주세요,남성,185.0,61.0,DOODLE LOGO HOODIE - BLACK,XL,5.0,라이브특가로 저렴히 구매했어요. 오버핏이라서 더 좋습니다 굿굿,보통이에요,보통이에요,보통이에요,보통이에요,XL,72.0,69.5,67.0,62.0,라이브특가로 저렴히 구매했어요 오버핏이라서 더 좋습니다 굿굿,"핏이라서 더 좋습니다 굿굿,오버핏이라서,핏이라서 더 좋습니다,오버핏,핏이라서 더 좋...",0,"가로 저렴히 구매했어요 오버핏이라서,라이브특가로","가로 저렴히 구매했어요 오버핏이라서,라이브특가로",0,1,0,0,0,0,0,1,0,1,0


## small,big 여기가 쓸것!!!!!!!!!!!!!!!!!

In [88]:
#small,big
def big_small(big,small,df,name_big_small):
    start = time.time()
    review = df[name_big_small+'_keyword']
    df[name_big_small + '_big'] = str('')
    df[name_big_small + '_small'] = str('')
    for i in tqdm(range(len(review))):
        # if i % 1000 ==0:
        #     print("{}번째 리뷰 완료(천 단위) : ".format(i))
        big_search = []
        small_search = []
        for j in big:
            if re.findall(j, review[i]):
                a = re.findall(j,review[i]) #키워드
                big_search.extend(a)
               
        for j in small:
            if re.findall(j, review[i]):
                a = re.findall(j,review[i]) #키워드
                small_search.extend(a)
            
               
        
        if len(big_search) != 0:
            df[name_big_small + '_big'][i] = 1
            
        else:
            df[name_big_small + '_big'][i] = 0
        
        if len(small_search) != 0:
            df[name_big_small + '_small'][i] = 1
            
        else:
            df[name_big_small + '_small'][i] = 0
        #time.sleep(0.2)
    
    #df.rename(columns = {'keyword_column_name':keyword_column_name},inplace=True)
    end = time.time()
    print("{:.5f} sec".format(end-start))
    return df[[name_big_small + '_big',name_big_small + '_small']]

In [92]:
name_big_small      = 'total'
total_big_small     = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'chongjang'
chongjang_big_small = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'arm'
arm_big_small       = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'chest'
chest_big_small     = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'shoulder'
shoulder_big_small  = big_small(big,small,df_keywords,name_big_small)

final_df            = df_keywords

100%|██████████| 20037/20037 [00:48<00:00, 413.72it/s]


48.43898 sec


100%|██████████| 20037/20037 [00:52<00:00, 384.14it/s]


52.16604 sec


100%|██████████| 20037/20037 [00:49<00:00, 402.30it/s]


49.81491 sec


100%|██████████| 20037/20037 [00:51<00:00, 387.44it/s]


51.72124 sec


100%|██████████| 20037/20037 [00:48<00:00, 413.61it/s]

48.44972 sec


In [91]:
#df_keywords.columns
df_keywords =df_keywords.drop(['total_big','total_small'],axis = 1)
df_keywords.columns

Index(['user', 'gender', 'height', 'weight', 'item', 'size', 'star', 'content',
       'size_eval', 'bright_eval', 'color_eval', 'thick_eval', 'cm', '총장',
       '어깨너비', '가슴단면', '소매길이', 'review', 'total_keyword', 'chongjang_keyword',
       'chest_keyword', 'shoulder_keyword', 'arm_keyword'],
      dtype='object')

In [93]:
print('전체big{},전체small{}'.format(final_df[final_df['total_big']!=0].shape,final_df[final_df['total_small']!=0].shape))
print('총장big{},총장small{}'.format(final_df[final_df['chongjang_big']!=0].shape,final_df[final_df['chongjang_small']!=0].shape))
print('가슴big{},가슴small{}'.format(final_df[final_df['chest_big']!=0].shape,final_df[final_df['chest_small']!=0].shape))
print('어깨big{},어깨small{}'.format(final_df[final_df['shoulder_big']!=0].shape,final_df[final_df['shoulder_small']!=0].shape))
print('소매big{},소매small{}'.format(final_df[final_df['arm_big']!=0].shape,final_df[final_df['arm_small']!=0].shape))


전체big(4737, 33),전체small(689, 33)
총장big(1380, 33),총장small(516, 33)
가슴big(163, 33),가슴small(47, 33)
어깨big(262, 33),어깨small(72, 33)
소매big(625, 33),소매small(86, 33)


In [97]:
test = final_df[final_df['chest_big']!=0]
test[['chest_keyword','chest_big','chest_small']].head(10)

,chest_keyword,chest_big,chest_small
243,"통 길이가 짧아서 시보리가,몸통,몸통 길이가 짧아서 시보리가,비해 몸통",1,1
724,"넓이 적당하고 길이도 길지,오버핏사이즈입니다 넓이",1,0
874,"가슴폭은 많이 크지않고,에 가슴",1,0
897,"통도 진짜 큰데,크고 통",1,0
1481,"가로 저렴히 구매했어요 오버핏이라서,라이브특가로",1,0
1616,"통통해서 큰거 입으면,잘했어요통통",1,0
1871,"통이 생각보다 크고,같네요팔통",1,0
2010,"통도 엄청 큽니다시보리도,팔통",1,0
2018,"통이 엄청 크네요,팔 통",1,0
2132,"가슴둘레 넉넉하고 귀엽습니다,예쁩니다 가슴,가슴둘레",1,0


In [98]:
test = final_df[final_df['chest_small']!=0]
test[['chest_keyword','chest_big','chest_small']].head(10)

,chest_keyword,chest_big,chest_small
243,"통 길이가 짧아서 시보리가,몸통,몸통 길이가 짧아서 시보리가,비해 몸통",1,1
886,"가슴둘레는 아주 살짝,맞는데 가슴,둘레는 아주 살짝,가슴둘레",0,1
1350,"둘레가 좀 좁은,허리 둘레",0,1
2981,"통입니다 생각보다 살짝,보통",0,1
3501,몸집이 작아서 다른,0,1
3567,"통은 커요 약간,짧은데 통",1,1
3914,"통목부분은 좁긴 해요,보통,가슴단면 입는데이 옷은,반팔입으면 가슴",1,1
3998,"통이 좁지 않아서,팔통",0,1
4204,"몸집이 작은 아이라,마르고 몸집",0,1
5885,"통이랑 상체 작은,통도 커서 팔,샀는데머리통,크고팔통",1,1


In [99]:
test = final_df[final_df['arm_big']!=0]
test[['arm_keyword','arm_big','arm_small']].head(10)

,arm_keyword,arm_big,arm_small
162,"소매길이 모두 적당합니다,어깨 소매",1,0
194,"기준에는 팔이,팔이 많이 길어요,기준에는 팔",1,0
199,"팔이길고 전체적인 길이감은,팔이길고 전체적인 길이감은",1,0
200,"팔이길고 전체적인 길이감은,팔이길고 전체적인 길이감은",1,0
234,"팔길이는 살짝 길지만,팔길이는 살짝 길지만,팔길이는 살짝 길지만",1,1
236,"팔이 좀 길지만 다른건,팔이 좀 길지만",1,0
243,"요팔길이는 좋은데 그에,않아요팔,팔길이는 좋은데 그에,않아요팔길이,팔길이는 좋은데 ...",1,0
256,"소매길이는 긴데 뭐,오버핏이에요 소매",1,0
357,"팔 길이는 완전 적당해요,길이구요 팔",1,0
429,"팔길이에 비해서 몸쪽,근데 팔길이,팔길이에 비해서 몸쪽,근데 팔,팔길이에 비해서 몸...",1,0


In [100]:
test = final_df[final_df['arm_small']!=0]
test[['arm_keyword','arm_big','arm_small']].head(10)

,arm_keyword,arm_big,arm_small
168,짧게나오고 소매,0,1
234,"팔길이는 살짝 길지만,팔길이는 살짝 길지만,팔길이는 살짝 길지만",1,1
1437,"소매가 쌀짝 널널한거,고민했었는데 소매",1,1
1787,"요팔 기장이 아주약간 짧지만,떨어져요팔,팔 기장이 아주약간 짧지만,떨어져요팔",0,1
1809,"팔부분이 좀 짧고,근데 팔,손에 가까워질수록 작아져요,삼각형이에요 손",0,1
1810,"팔이 조금 짧나싶은 느낌은,팔이 조금 짧나싶은",0,1
2773,"손목부분 너무 타이트하지,크다는점과 손목,손목부분 너무 타이트하지,크다는점과 손",1,1
3799,"손목조임도 타이트해서 마음어,밑단 손목,손목조임도 타이트해서 마음어,밑단 손",0,1
3998,"팔통이 좁지 않아서,짱짱하고 팔",0,1
4645,"요팔이 살짝 길긴한데,편해요팔,팔이 살짝 길긴한데 그것도,편해요팔이,팔이 살짝 길긴...",1,1


# 안써!!!!!!

In [128]:
#쓸지말지 고민
# #최종 사이즈 사전 완성
def final_sizedict(size_figure_list):
    all_big = {}
    all_small = {}
    for i in size_figure_list:
        all_big[i] = big
    
    for i in size_figure_list:
        all_small[i] = small
    
    return all_big,all_small

In [129]:
total_big,total_small = final_sizedict(total)
chongjang_big,chongjang_small = final_sizedict(chongjang)
shoulder_big,shoulder_small = final_sizedict(shoulder)
arm_big,arm_small = final_sizedict(arm)
chest_big,chest_small = final_sizedict(chest)


In [ ]:
# # #최종 사이즈 사전 완성
# # def final_sizedict(size_figure_list):
# #     all_big = {}
# #     all_small = {}
# #     for i in size_figure_list:
# #         all_big[i] = big
    
# #     for i in size_figure_list:
# #         all_small[i] = small
    
# #     return all_big,all_small
# # %%
# df = crawlingdataconcat()
# df = crawlingdataprocessing(df)
# df = add_reviewcol(df)
# #df.to_csv('data/hood_1to20.csv', encoding="UTF-8", index=False) #파일로 저장
# # %%
# total_big,total_small           = final_sizedict(total)
# chongjang_big,chongjang_small   = final_sizedict(chongjang)
# shoulder_big,shoulder_small     = final_sizedict(shoulder)
# arm_big,arm_small               = final_sizedict(arm)
# chest_big,chest_small           = final_sizedict(chest)

In [18]:
print('null값모두삭제:{}'.format(df.dropna(axis = 0).shape))
print('성별,키,몸무게 null 삭제:{}'.format(df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()].shape))
print('성별,키,몸무게 nonnull & 사이즈 하나라도 nonnull:{}'.format(df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())].shape))
print('성별,키,몸무게 null:{}'.format(df[df['gender'].isnull()&df['height'].isnull()&df['weight'].isnull()].shape))
print('총장,어깨너비,가슴단면,소매길이4개 전부 null:{}'.format(df[df['총장'].isnull()&
   df['어깨너비'].isnull()&df['가슴단면'].isnull()&
   df['소매길이'].isnull()].shape))
print('총장,어깨너비,가슴단면,소매길이 하나라도 null:{}'.format(df[df['총장'].isnull()|
   df['어깨너비'].isnull()|df['가슴단면'].isnull()|
   df['소매길이'].isnull()].shape))



null값모두삭제:(19511, 18)
성별,키,몸무게 null 삭제:(33350, 18)
성별,키,몸무게 nonnull & 사이즈 하나라도 nonnull:(20467, 18)
성별,키,몸무게 null:(2402, 18)
총장,어깨너비,가슴단면,소매길이4개 전부 null:(13847, 18)
총장,어깨너비,가슴단면,소매길이 하나라도 null:(14967, 18)


In [20]:
df_figure = df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())]
df_figure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20467 entries, 100 to 35737
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   20467 non-null  int64  
 1   user         20467 non-null  object 
 2   gender       20467 non-null  object 
 3   height       20467 non-null  object 
 4   weight       20467 non-null  object 
 5   item         20467 non-null  object 
 6   size         20467 non-null  object 
 7   star         20467 non-null  float64
 8   content      20467 non-null  object 
 9   size_eval    20467 non-null  object 
 10  bright_eval  20467 non-null  object 
 11  color_eval   20467 non-null  object 
 12  thick_eval   20467 non-null  object 
 13  cm           20467 non-null  object 
 14  총장           20467 non-null  float64
 15  어깨너비         19611 non-null  float64
 16  가슴단면         20467 non-null  float64
 17  소매길이         20366 non-null  float64
dtypes: float64(5), int64(1), object(12)
memory u